## Importing Libraries

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import torch

## Virtual Try-On System

In [2]:
def segmentor(image_path):

    # Load the image
    img = cv2.imread(image_path)

    # Preprocessing
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)  # Apply Gaussian blur

    # Segmentation (using GrabCut)
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (50, 50, 450, 450)  # Initial rectangle for GrabCut
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    img_segmented = img * mask2[:, :, np.newaxis]

    # Clothing detection (using a pre-trained YOLOv5 model)
    # model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    # results = model(img_segmented)

    cv2.imshow("Segmented Image", img_segmented)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return img_segmented

# Initialize MediaPipe's pose detection model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def try_on_clothing(user_image, clothing_image):

    # Create a mask for the user's body
    body_mask = np.zeros(user_image.shape[:2], np.uint8)
    
    # Initialize the pose detection model
    pose_model = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8)
    
    # Process the user image to get pose landmarks
    pose_results = pose_model.process(cv2.cvtColor(user_image, cv2.COLOR_BGR2RGB))

    # Extract landmarks for left and right shoulders, and left and right hips
    left_shoulder_user = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder_user = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_foot_user = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]
    right_foot_user = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]

    # Convert the landmarks to image coordinates
    image_height, image_width, _ = user_image.shape
    left_shoulder_user_x, left_shoulder_user_y = int(left_shoulder_user.x * image_width), int(left_shoulder_user.y * image_height)
    right_shoulder_user_x, right_shoulder_user_y = int(right_shoulder_user.x * image_width), int(right_shoulder_user.y * image_height)
    left_foot_user_x, left_foot_user_y = int(left_foot_user.x * image_width), int(left_foot_user.y * image_height)
    right_foot_user_x, right_foot_user_y = int(right_foot_user.x * image_width), int(right_foot_user.y * image_height)

    # Calculate the size of the clothing image based on the user's pose
    shoulder_width_user = abs(right_shoulder_user_x - left_shoulder_user_x)
    foot_width_user = abs(right_foot_user_x - left_foot_user_x)
    
    # Resize the clothing image to fit the user's torso
    clothing_image_resized = cv2.resize(clothing_image, (shoulder_width_user, foot_width_user))

    # Define the corners of the region to be replaced with the clothing image
    # source points
    src_pts = np.array([[0, 0],
                        [shoulder_width_user, 0],
                        [shoulder_width_user, foot_width_user],
                        [0, foot_width_user]], dtype=np.float32)
    
    # Destination points
    dst_pts = np.array([[left_shoulder_user_x, left_shoulder_user_y],
                        [right_shoulder_user_x, right_shoulder_user_y],
                        [right_foot_user_x, right_foot_user_y],
                        [left_foot_user_x, left_foot_user_y]], dtype=np.float32)

    # Get the perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # Warp the clothing image onto the user image
    warped_clothing = cv2.warpPerspective(clothing_image_resized, M, (image_width, image_height))

    # Create a mask of the region to be replaced with the clothing image
    mask = np.zeros_like(user_image)
    cv2.fillPoly(mask, [np.int32(dst_pts)], (255, 255, 255))

    # Invert the mask
    mask_inv = cv2.bitwise_not(mask)

    # Extract the region from the user image that will be replaced
    user_image_region = cv2.bitwise_and(user_image, mask_inv)

    # Blend the clothing image with the user image
    final_image = cv2.add(warped_clothing, user_image_region)

    # Display the final image
    cv2.imshow("Virtual Try-On", final_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [3]:
user_image_path = r"C:\Users\shish\OneDrive\Documents\GitHub\Virtual_wardrobe_recommender_system\lady.jpeg"
clothing_image_path = r"C:\Users\shish\OneDrive\Documents\GitHub\Virtual_wardrobe_recommender_system\clothing.jpg"

user_image = segmentor(user_image_path)
clothing_image = segmentor(clothing_image_path)

try_on_clothing(user_image, clothing_image)